In [20]:
%cd /code

/code


In [21]:
from source.service.datasets import DATA
from source.library.scraping import scrape_all_urls#, scrape_urls

# This is needed because openai.text_completion calls asynchronous functions but
# Jupyter is already running its own event loop.
import nest_asyncio
nest_asyncio.apply()

In [22]:
langchain_doc_chunks = DATA.langchain_doc_chunks.load()
print(len(langchain_doc_chunks))
print(langchain_doc_chunks[0])

11060
('https://python.langchain.com/en/latest', 'Welcome to LangChain # LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model via an API, but will also: Be data-aware : connect a language model to other sources of data Be agentic : allow a language model to interact with its environment The LangChain framework')


In [23]:
from langchain.vectorstores import Chroma
persist_directory = '/code/.vectordb/'

In [24]:
vectordb = Chroma(persist_directory=persist_directory)

In [25]:
vectordb.similarity_search(query="What is Langchain?", k=5)

[Document(page_content="Human: For LangChain! Have you heard of it? AI:  No, I haven't heard of LangChain. Can you tell me more about it? Human: Haha nope, although a lot of people confuse it for that AI: > Finished chain. ' Oh, okay. What is LangChain?'", metadata={'url': 'https://python.langchain.com/en/latest/modules/memory/types/summary_buffer.html'}),
 Document(page_content='of all related terms, papers, methods, etc. Whether implemented in LangChain or not!\\nGallery: A collection of our favorite projects that use LangChain. Useful for finding inspiration or seeing how things were done in other applications.\\nDeployments: A collection of instructions, code snippets, and template repositories for deploying LangChain apps.\\nDiscord: Join us on our Discord to discuss all', metadata={'url': 'https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/sitemap.html'}),
 Document(page_content='of all related terms, papers, methods, etc. Whether implemented in Lang

---

In [31]:
# from langchain.chains import VectorDBQA
from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp

# N_CONTEXT = 1024
# model_path = './models/ggml-alpaca-7b-q4.bin'

N_CONTEXT = 2048
model_path = './models/ggml-vicuna-13b-1.1-q4_0.bin'

llm = LlamaCpp(model_path=model_path, temperature=0.0, n_ctx=N_CONTEXT)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    verbose=False,
)

llama.cpp: loading model from ./models/ggml-vicuna-13b-1.1-q4_0.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 4 (mostly Q4_1, some F16)
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =  73.73 KB
llama_model_load_internal: mem required  = 9807.47 MB (+ 1608.00 MB per state)
llama_init_from_file: kv self size  = 1600.00 MB
AVX = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 0 | NEON = 1 | ARM_FMA = 1 | F16C = 0 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 0 | VSX = 0 | 


---

In [32]:
question = "What is LangChain?"
# print("Base-Model:")
# print(llm(prompt=f"Question: {question}? Answer: "))
print("\nContext-Model")
context_answer = qa.run(question)
print(context_answer)


Context-Model


> Entering new RetrievalQA chain...

> Finished chain.
 LangChain is a platform for building and deploying blockchain-based applications. It provides a set of tools, libraries, and services to help developers create decentralized applications that can be deployed on various blockchain networks.


In [47]:
print(qa.json())

{"memory": null, "callbacks": null, "callback_manager": null, "verbose": true, "combine_documents_chain": {"memory": null, "callbacks": null, "callback_manager": null, "verbose": false, "input_key": "input_documents", "output_key": "output_text", "llm_chain": {"memory": null, "callbacks": null, "callback_manager": null, "verbose": false, "prompt": {"input_variables": ["context", "question"], "output_parser": null, "partial_variables": {}, "template": "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:", "template_format": "f-string", "validate_template": true, "_type": "prompt"}, "llm": {"model_path": "./models/ggml-vicuna-13b-1.1-q4_0.bin", "suffix": null, "max_tokens": 256, "temperature": 0.0, "top_p": 0.95, "logprobs": null, "echo": false, "stop_sequences": [], "repeat_penalty": 1.1, "top_k": 40, "_type": "llama.cpp"}, 

In [48]:
qa.combine_documents_chain.llm_chain.prompt.template

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"

In [49]:
vectordb.similarity_search(query="What is Langchain?", k=5)

[Document(page_content="Human: For LangChain! Have you heard of it? AI:  No, I haven't heard of LangChain. Can you tell me more about it? Human: Haha nope, although a lot of people confuse it for that AI: > Finished chain. ' Oh, okay. What is LangChain?'", metadata={'url': 'https://python.langchain.com/en/latest/modules/memory/types/summary_buffer.html'}),
 Document(page_content='of all related terms, papers, methods, etc. Whether implemented in LangChain or not!\\nGallery: A collection of our favorite projects that use LangChain. Useful for finding inspiration or seeing how things were done in other applications.\\nDeployments: A collection of instructions, code snippets, and template repositories for deploying LangChain apps.\\nDiscord: Join us on our Discord to discuss all', metadata={'url': 'https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/sitemap.html'}),
 Document(page_content='of all related terms, papers, methods, etc. Whether implemented in Lang

---

In [50]:
question = "How do I use prompt templates for creating a chat bot?"
print("Base-Model:")
print(llm(prompt=f"Question: {question}? Answer: "))
print("\nContext-Model")
context_answer = qa.run(question)
print(context_answer)

Base-Model:


To use prompt templates for creating a chatbot, you can follow these steps:

1. Define the intents of your chatbot - i.e., what tasks or questions the chatbot should be able to handle.
2. Create a list of prompts for each intent - i.e., a list of messages that the chatbot can send in response to different user inputs.
3. Use a natural language processing (NLP) library, such as spaCy or NLTK, to extract keywords and phrases from user input and match them against the prompts for each intent.
4. Select the most relevant prompt based on the user input and send it back to the user in the form of a message.
5. Continuously monitor the chatbot's interactions with users and update the prompt templates as needed to improve its performance.

Context-Model


> Entering new RetrievalQA chain...

> Finished chain.
 You can use the PromptTemplates module to create custom prompts for your chat bot by using the Chat Prompt Template example selectors and output parsers. The Chat Prompt Te

---

In [ ]:
question = "How do I use prompt templates for creating a chat bot?"
print("Base-Model:")
base_answer = llm(prompt=f"Question: {question}? Answer: ")
print(base_answer)
print("\nContext-Model")
context_answer = qa.run(question)
print(context_answer)

In [ ]:
def print_similarity_search(results):
    pass



In [ ]:
results = vectordb.similarity_search(query=question, k=5)
for r in results:
    print(r.metadata['url'])
    print(r.page_content)

In [ ]:
vectordb.similarity_search(query=base_answer, k=5)

In [ ]:
vectordb.similarity_search(query=context_answer, k=5)

In [ ]:
question = "What is the map_reduce chain type?"
print("Base-Model:")
base_answer = llm(prompt=f"Question: {question}? Answer: ")
print(base_answer)
print("Context-Model:")
context_answer = qa.run(question)
print(context_answer)

---

# Appendix

In [ ]:
from bs4 import BeautifulSoup
import numpy as np
import chromadb

client = chromadb.Client()

collection = client.create_collection('sample_collection')

documents = []
metadatas = []

for url, chunk in langchain_doc_chunks:
        documents.append(chunk)
        metadatas.append({'url': url})

collection.add(
    documents=documents, # we embed for you, or bring your own
    metadatas=metadatas, # filter on arbitrary metadata!
    ids=[str(x) for x in range(len(documents))], # must be unique for each doc 
)

print(np.mean([len(x) for x in documents]))
print(np.max([len(x) for x in documents]))

In [ ]:
print(documents[0:10])

In [ ]:
results = collection.query(
    query_texts="What is a document loader?",
    n_results=5,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)
print(len(results['documents'][0]))

In [ ]:
results